In [188]:
import numpy as np
import pandas as pd 
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
import pysam

In [433]:
reverse_complement('CCTGACACTGCATCCTCGTC')

'GACGAGGATGCAGTGTCAGG'

In [ ]:
2.BEST1_Y284C_ABE8e: 

F: CCTGACACTGCATCCTCGTC 

R: CCCCTGGAGCATCCTGATTT 

CCCCTGGAGCATCCTGATTTCAGGGTTCCCACCTAGCCCTTTGCTACCACATCCTCCTCCTCCTCCCAGGTGGTGACTGTGGCGGTGTACAGCTTCTTCCTGACTTGTCTAGTTGGGCGGCAGTTTCTGAACCCAGCCAAGGCCTACCCTGGCCATGAGCTGGACCTCGTTGTGCCCGTCTTCACGTTCCTGCAGTTCTTCTTCTATGTTGGCTGGCTGAAGGTGGGCCTCTCCAGGGCCCTGCTGGGCTGGAGGCATGGCCAGAGGGGTCATGGCCAGCAGCTGCCTGAGACGAGGATGCAGTGTCAGG

In [183]:
def merge_and_trim_sequences(seq1, seq2, umi1, umi2):
    # Align the two sequences using local alignment
    alignments = pairwise2.align.localms(seq1, seq2, 2, -1, -3, -0.5, one_alignment_only=True)
    aligned_seq1, aligned_seq2, score, start, end = alignments[0]

    # Initialize the consensus sequence with the first aligned sequence
    consensus = list(aligned_seq1)

    # Merge the aligned sequences into the consensus sequence
    for i, (base1, base2) in enumerate(zip(aligned_seq1, aligned_seq2)):
        if base1 == '-':
            consensus[i] = base2
        elif base2 == '-':
            continue
        elif base1 == 'N' and base2 != 'N':
            consensus[i] = base2
        elif base2 == 'N' and base1 != 'N':
            consensus[i] = base1
        elif base1 != base2:
            # Choose the base from seq2 if a mismatch occurs, unless seq2 contains 'N'
            consensus[i] = base2 if base2 != 'N' else base1

    # Join consensus list into string and remove gaps
    consensus = ''.join(consensus).replace('-', '')

    # Find UMI1 and remove everything to the left
    umi1_index = consensus.find(umi1)
    #if umi1_index != -1:
    #    consensus = consensus[umi1_index + len(umi1):]

    # Find UMI2 and remove everything to the right (reverse complement)
    

    umi2_rc = reverse_complement(umi2)
    umi2_index = consensus.rfind(umi2_rc)
    if (umi2_index != -1) and (umi1_index != -1):
        consensus = consensus[umi1_index + len(umi1):]
        consensus = consensus[:umi2_index]
        

    return consensus

def reverse_complement(seq):
    complement = str.maketrans("ATGC", "TACG")
    return seq.translate(complement)[::-1]

In [423]:
def extract_barcode(consensus, vector, barcode_length=12):
    # Align the consensus sequence to the known vector sequence
    alignments = pairwise2.align.localms(consensus, vector, 2, -1, -3, -0.5, one_alignment_only=True)
    if alignments:
        aligned_consensus_seq, aligned_vector_seq, score, start, end = alignments[0]
        barcode_start = end
        barcode_end = barcode_start + barcode_length
        barcode = aligned_consensus_seq[barcode_start:barcode_end]
        return barcode, score 
    return "", 0

pre_barcode_vector1 = 'GTACCAAGGTCGGGCAGGAA' #9nt after 
pre_barcode_vector2 = 'GTTTAAGAGCTAAGCTGGAA' #12nt after 


Reference = 'NNNNNNNTGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGGAANNNNNNNNNGAGGGCCTATTTCCCATGATTCCTTCATATTTGCATATACGATACAAGGCTGTTAGAGAGATAATTAGAATTAATTTGACTGTAAACACAAAGATATTAGTACAAAATACGTGACGTAGAAAGTAATAATTTCTTGGGTAGTTTGCAGTTTTAAAATTATGTTTTAAAATGGACTATCATATGCTTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCTTGTGGAAAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAANNNNNNNNNNNNAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTATGACTNNNNNNN'

In [307]:
"NNNNNNNTGACTaaccaattcagtcgactggatccggtaccaaggtcgggcaggaaNNNNNNNNNgagggcctatttcccatgattccttcatatttgcatatacgatacaaggctgttagagagataattagaattaatttgactgtaaacacaaagatattagtacaaaatacgtgacgtagaaagtaataatttcttgggtagtttgcagttttaaaattatgttttaaaatggactatcatatgcttaccgtaacttgaaagtatttcgatttcttggctttatatatcttgtggaaaggacgaaacaccGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAANNNNNNNNNNNNAAAAAGTGGCACCGAGTCGGTGCgcattacagtacgaagatgcatagctttttttaagcttgggccgctcgaggtacctctctaTGACTNNNNNNN".upper()

'NNNNNNNTGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGGAANNNNNNNNNGAGGGCCTATTTCCCATGATTCCTTCATATTTGCATATACGATACAAGGCTGTTAGAGAGATAATTAGAATTAATTTGACTGTAAACACAAAGATATTAGTACAAAATACGTGACGTAGAAAGTAATAATTTCTTGGGTAGTTTGCAGTTTTAAAATTATGTTTTAAAATGGACTATCATATGCTTACCGTAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCTTGTGGAAAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAANNNNNNNNNNNNAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTATGACTNNNNNNN'

In [411]:
def extract_Promoter(consensus, barcode1, guide='GGCCCAGACTGAGCACGTGA'):
    
    start_index = consensus.find(barcode1)+len(barcode1) #Guide Index
    
    #Now get alignment of the guide
    alignments = pairwise2.align.localms(consensus, guide, 2, -1, -3, -0.5, one_alignment_only=True)
    aligned_consensus_seq, aligned_vector_seq, score, start_guide, end_guide = alignments[0]
    if score < 32: #Indicates vector is not mapping anywhere as well as it should
        return 'NG'
    elif start_index >= start_guide:
        return 'NB'
    else:    
    #Return the sequence between the Barcode and the guide
        return consensus[start_index:start_guide]
    

In [352]:
def process_bam_file(input_bam):
    samfile = pysam.AlignmentFile(input_bam, "rb")
    Barcodes_Promoter = []
    
    # Track the last processed read name to detect and skip additional occurrences
    last_processed_read_name = None
    pending_read = None

    # Read through the BAM file and process reads in pairs
    try:
        for read in samfile:
            if read.query_name == last_processed_read_name:
                # Skip this read if we have already processed a pair with this read name
                continue

            if pending_read is None:
                # If there's no pending read, store the current read
                pending_read = read
            else:
                # We have a pair to process
                read1, read2 = pending_read, read

                # Reset the pending read
                pending_read = None

                # Process the pair
                UMI = read1.query_name.split('_')[-1]
                umi1, umi2 = UMI[:7], UMI[7:14]
                consensus_seq = merge_and_trim_sequences(read1.seq, read2.seq, umi1, umi2)
                barcode1, score1 = extract_barcode(consensus_seq, pre_barcode_vector1, barcode_length=9)
                barcode2, score2 = extract_barcode(consensus_seq, pre_barcode_vector2, barcode_length=12)
                if (score1 or score2) < 32: #Indicates vector is not mapping anywhere as well as it should
                    #Excluded from the analysis
                    #Either no vector or no barcode. 
                    Barcodes_Promoter.append((UMI,len(consensus_seq), '', '', 'NV'))
                    continue
                elif (Reference.find(barcode1) != -1) or (Reference.find(barcode2) != -1):
                    Barcodes_Promoter.append((UMI,len(consensus_seq), '', '', 'NB'))
                    continue
                    
                Promoter = extract_Promoter(consensus_seq, barcode1, guide='GGCCCAGACTGAGCACGTGA')
                Barcodes_Promoter.append((UMI,len(consensus_seq), barcode1, barcode2, Promoter))

                # Update the last processed read name
                last_processed_read_name = read1.query_name

    except StopIteration:
        # End of file reached
        pass
    
    samfile.close()
    return Barcodes_Promoter

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Process BAM file and export to CSV.")
    parser.add_argument("input_bam", help="Input BAM file path")
    parser.add_argument("output_file", help="Output CSV file path")
    args = parser.parse_args()
    results = process_bam_file(args.input_bam)
    df = pd.DataFrame(results, columns = ['UMI','Length', 'Barcode1', 'Barcode2', 'U6'])
    df.to_csv(args.output_file,  index = False, header = False)


In [412]:
# Example usage
input_bam = "little.bam"
results = process_bam_file(input_bam)
#df = pd.DataFrame(results, columns = ['UMI','Length', 'Barcode1', 'Barcode2', 'U6'])
df.to_csv('OUTPUT.csv', index = False, header = False)
#for item in results:
#    print(item)

In [413]:
df = pd.DataFrame(results, columns = ['UMI','Length', 'Barcode1', 'Barcode2', 'U6'])

In [422]:
df.to_csv('OUTPUT.csv', index = False, header = False)

In [262]:
def process_bam_file_temp(input_bam):
    samfile = pysam.AlignmentFile(input_bam, "rb")
    Barcodes_Promoter = []
    
    # Track the last processed read name to detect and skip additional occurrences
    last_processed_read_name = None
    pending_read = None

    # Read through the BAM file and process reads in pairs
    try:
        for read in samfile:
            if read.query_name == last_processed_read_name:
                # Skip this read if we have already processed a pair with this read name
                continue

            if pending_read is None:
                # If there's no pending read, store the current read
                pending_read = read
            else:
                # We have a pair to process
                read1, read2 = pending_read, read

                # Reset the pending read
                pending_read = None

                # Process the pair
                UMI = read1.query_name.split('_')[-1]
                umi1, umi2 = UMI[:7], UMI[7:14]
                consensus_seq = merge_and_trim_sequences(read1.seq, read2.seq, umi1, umi2)
                bc, score = extract_barcode_temp(consensus_seq, pre_barcode_vector1, barcode_length=9 )
                bc, score1 = extract_barcode_temp(consensus_seq, pre_barcode_vector2, barcode_length=12 )
                print(score,score1, UMI)
    except StopIteration:
        # End of file reached
        pass
    
    samfile.close()
    return Barcodes_Promoter
    

In [365]:
read1='AACTCGACACTCTTTCCCTACACGACGCTCTTCCGATCTACATTGATGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGTGCAGGAATACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAAGGCCATCTTCACAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA'
read2='TGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGGAATACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAAGGCCATCTTCACAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCAGCTTGGA'

In [366]:
# Extract UMIs from read name
query_name = 'M01605:44:000000000-LHM7F:1:1101:2206:12290_ACATTGATCCAAGC'
UMI = query_name.split('_')[-1]
umi1, umi2 = UMI[:7], UMI[7:14]

# Sequence merging and trimming
consensus_seq = merge_and_trim_sequences(read1, read2, umi1, umi2)

# Extracting barcodes
barcode1, score1 = extract_barcode(consensus_seq, pre_barcode_vector1, barcode_length=9)
barcode2 , score2= extract_barcode(consensus_seq, pre_barcode_vector2, barcode_length=12)

# Extracting promoter
Promoter = extract_Promoter(consensus_seq, barcode1, guide='GGCCCAGACTGAGCACGTGA')

In [367]:
alignments = pairwise2.align.localms(read1, read2, 2, -1, -3, -0.5, one_alignment_only=True)
aligned_seq1, aligned_seq2, score, start, end = alignments[0]

# Initialize the consensus sequence with the first aligned sequence
consensus = list(aligned_seq1)

# Merge the aligned sequences into the consensus sequence
for i, (base1, base2) in enumerate(zip(aligned_seq1, aligned_seq2)):
    if base1 == '-':
        consensus[i] = base2
    elif base2 == '-':
        continue
    elif base1 == 'N' and base2 != 'N':
        consensus[i] = base2
    elif base2 == 'N' and base1 != 'N':
        consensus[i] = base1
    elif base1 != base2:
        # Choose the base from seq2 if a mismatch occurs, unless seq2 contains 'N'
        consensus[i] = base2 if base2 != 'N' else base1

# Join consensus list into string and remove gaps
consensus = ''.join(consensus).replace('-', '')


In [409]:

    
start_index = consensus.find(barcode1)+len(barcode1) #Guide Index

#Now get alignment of the guide
alignments = pairwise2.align.localms(consensus, guide, 2, -1, -3, -0.5, one_alignment_only=True)
aligned_consensus_seq, aligned_vector_seq, score, start_guide, end_guide = alignments[0]

#Return the sequence between the Barcode and the guide

consensus[start_index:start_guide]
    

''

In [382]:
alignments = pairwise2.align.localms(consensus, guide, 2, -1, -3, -0.5, one_alignment_only=True)
aligned_consensus_seq, aligned_vector_seq, score, start_guide, end_guide = alignments[0]
for alignment in alignments:
    print(format_alignment(*alignment))

102 GGCCCAGACTGAGCACGTGA
    ||||||||||||||||||||
  1 GGCCCAGACTGAGCACGTGA
  Score=40



In [368]:
def extract_barcode_temp(consensus, vector, barcode_length=12):
    # Align the consensus sequence to the known vector sequence
    alignments = pairwise2.align.localms(consensus, vector, 2, -1, -3, -0.5, one_alignment_only=True)
    if alignments:
        aligned_consensus_seq, aligned_vector_seq, score, start, end = alignments[0]
        barcode_start = end
        barcode_end = barcode_start + barcode_length
        barcode = aligned_consensus_seq[barcode_start:barcode_end]
        return barcode, score
    return ""

In [369]:
alignments = pairwise2.align.localms(consensus, pre_barcode_vector1, 2, -1, -3, -0.5, one_alignment_only=True)

for alignment in alignments:
    print(format_alignment(*alignment))

76 GTACCAAGGTCGGGCAGGAA
   ||||||||||||||||||||
 1 GTACCAAGGTCGGGCAGGAA
  Score=40



In [198]:
# Find UMI1 and remove everything to the left
umi1_index = consensus.find(umi1)
if umi1_index != -1:
    consensus = consensus[umi1_index + len(umi1):]

# Find UMI2 and remove everything to the right (reverse complement)


umi2_rc = reverse_complement(umi2)
umi2_index = consensus.rfind(umi2_rc)
if umi2_index != -1:
    consensus = consensus[:umi2_index]

NNNNNNNTGACTaaccaattcagtcgactggatccggtaccaaggtcgggcaggaaNNNNNNNNNgagggcctatttcccatgattccttcatatttgcatatacgatacaaggctgttagagagataattagaattaatttgactgtaaacacaaagatattagtacaaaatacgtgacgtagaaagtaataatttcttgggtagtttgcagttttaaaattatgttttaaaatggactatcatatgcttaccgtaacttgaaagtatttcgatttcttggctttatatatcttgtggaaaggacgaaacaccGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAANNNNNNNNNNNNAAAAAGTGGCACCGAGTCGGTGCgcattacagtacgaagatgcatagctttttttaagcttgggccgctcgaggtacctctctaTGACTNNNNNNN


Vector1: aaccaattcagtcgactggatccggtaccaaggtcgggcaggaa

9nt Barcode:

U6: gagggcctatttcccatgattccttcatatttgcatatacgatacaaggctgttagagagataattagaattaatttgactgtaaacacaaagatattagtacaaaatacgtgacgtagaaagtaataatttcttgggtagtttgcagttttaaaattatgttttaaaatggactatcatatgcttaccgtaacttgaaagtatttcgatttcttggctttatatatcttgtggaaaggacgaaacacc

Guide: 
ggcccagactgagcacgtga

spacer: GTTTAAGAGCTAAGCTGGAA

linker-terminator-vector: AAAAAGTGGCACCGAGTCGGTGCgcattacagtacgaagatgcatagctttttttaagcttgggccgctcgaggtacctctctaTGACT